<a href="https://colab.research.google.com/github/talkin24/jaxflax_lab/blob/main/Flax_The_Sharp_Bits.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Flax는 JAX의 모든 기능을 제공합니다. 그리고 JAX를 사용할 때와 마찬가지로 Flax로 작업할 때 경험할 수 있는 특정 "날카로운 부분"이 있습니다.

먼저 Flax를 설치 및/또는 업데이트합니다:

In [1]:
! pip install -qq flax

## `flax.linen.Dropout` layer and randomness

### TL;DR?

드롭아웃이 있는 모델(`Flax Module`에서 서브클래싱된)에서 작업할 때는 포워드 패스 중에만 `'dropout'` PRNG키를 추가하세요.

1. `jax.random.split()`으로 시작하여 `'params'` 및 `'dropout'`에 대한 PRNG 키를 명시적으로 생성합니다.

2. 모델에 `flax.linen.Dropout` 레이어를 추가합니다(Flax `Module`에서 서브클래스화).

3. 모델을 초기화할 때(`flax.linen.init()`), "단순한" 모델에서처럼 `'params'` 키만 전달하면 되므로 추가 `'dropout'` PRNG 키를 전달할 필요가 없습니다.

4. `flax.linen.apply()`를 사용하여 포워드 패스를 전달하는 동안 `rngs={'dropout': dropout_key}`를 전달합니다.

아래에서 전체 예제를 확인하세요.

### Why this works

- 내부적으로 `flax.linen.Dropout`은 `flax.linen.Module.make_rng`를 사용하여 드롭아웃용 키를 생성합니다.

- `make_rng`가 호출될 때마다(이 경우 `Dropout`에서 암시적으로 수행됨) 메인/루트 PRNG 키에서 분할된 새 PRNG 키가 생성됩니다.

- `make_rng`는 여전히 완전한 재현성을 보장합니다.

### Background

드롭아웃 확률 정규화 기법은 네트워크에서 숨겨진 단위와 보이는 단위를 무작위로 제거합니다. 드롭아웃은 무작위 연산이므로 PRNG 상태가 필요하며, Flax(JAX와 마찬가지로)는 분할 가능한 Threefry PRNG를 사용합니다.



*참고: JAX에는 PRNG 키를 제공하는 명시적인 방법이 있다는 것을 기억하세요. 
주요 PRNG 상태(예: key = `jax.random.PRNGKey(seed=0))`를 `key, subkey = jax.random.split(key)`를 사용하여 여러 개의 새 PRNG 키로 포크할 수 있습니다.*


Flax는 Flax `Module`의 `flax.linen.Module.make_rng` 헬퍼 함수를 통해 PRNG 키 스트림을 처리하는 암시적인 방법을 제공합니다. 이 함수를 사용하면 Flax `Module`(또는 그 하위 `Module`)의 코드가 "PRNG 키를 가져올 수 있습니다." `make_rng`는 호출할 때마다 고유한 키를 제공하도록 보장합니다.

### Example

각 Flax PRNG 스트림에는 이름이 있다는 것을 기억하세요. 아래 예시에서는 매개변수를 초기화하기 위해 `'params'` 스트림과 `'dropout'` 스트림을 사용합니다. `flax.linen.init()`에 제공된 PRNG 키는 `'params'` PRNG 키 스트림을 시드하는 키입니다. 포워드 패스(드롭아웃 포함) 중에 PRNG 키를 가져오려면 `Module.apply()`를 호출할 때 해당 스트림(`'dropout'`)을 시딩할 PRNG 키를 제공하세요.

In [2]:
# Setup.
import jax
import jax.numpy as jnp
import flax.linen as nn


In [3]:
# Randomness.
seed = 0
root_key = jax.random.PRNGKey(seed=seed)
main_key, params_key, dropout_key = jax.random.split(key=root_key, num=3)

# A simple network.
class MyModel(nn.Module):
  num_neurons: int
  training: bool
  @nn.compact
  def __call__(self, x):
    x = nn.Dense(self.num_neurons)(x)
    # Set the dropout layer with a rate of 50% .
    # When the `deterministic` flag is `True`, dropout is turned off.
    x = nn.Dropout(rate=0.5, deterministic=not self.training)(x)
    return x

# Instantiate `MyModel` (you don't need to set `training=True` to
# avoid performing the forward pass computation).
my_model = MyModel(num_neurons=3, training=False)

x = jax.random.uniform(key=main_key, shape=(3, 4, 4))

# Initialize with `flax.linen.init()`.
# The `params_key` is equivalent to a dictionary of PRNGs.
# (Here, you are providing only one PRNG key.) 
variables = my_model.init(params_key, x)

# Perform the forward pass with `flax.linen.apply()`.
y = my_model.apply(variables, x, rngs={'dropout': dropout_key})